In [1]:
%run 'connect.ipynb'
cw.scope_logger.setLevel(cw.logging.WARNING) # turn warnings back on

In [2]:
scope.trigger.sequencer_enabled = False

In [3]:
scope.default_setup()

scope.adc.clip_errors_disabled           changed from True                      to False                    
scope.adc.lo_gain_errors_disabled        changed from True                      to False                    


In [25]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [26]:
cycles = scope.adc.trig_count
#cycles = 31864

In [27]:
cycles

31864

In [62]:
target.simpleserial_last_read

'r7DF76B0C1AB899B33E42F047B91B546F\n'

In [64]:
target.simpleserial_last_read[:4]

'r7DF'

In [29]:
scope.trigger.sequencer_enabled = True
scope.trigger.num_triggers = 2

scope.trigger.module[0] = 'basic'
scope.trigger.triggers[0] = 'tio4'

scope.trigger.module[1] = 'UART'
scope.trigger.triggers[1] = 'tio1'

In [30]:
scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 38400
scope.UARTTrigger.set_pattern_match(0, 'r7DF')
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0]

In [39]:
scope.trigger

sequencer_enabled      = True
max_sequenced_triggers = 2
num_triggers           = 2
sad_always_active      = False
module                 = SequenceTriggerList(['basic', 'UART'])
triggers               = SequenceTriggerList(['tio4', 'tio1'])
window_start           = SequenceTriggerList([0])
window_end             = SequenceTriggerList([0])
sequence trigger #0 = 
    module   = basic
    triggers = tio4
sequence trigger #1 = 
    module       = UART
    triggers     = tio1
    window_start = 0
    window_end   = 0

In [46]:
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [47]:
cycles

31864

In [48]:
scope.trigger.window_start[0] = cycles*2
scope.trigger.window_end[0] = 0
failtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12
(ChipWhisperer Scope WARNING|File __init__.py:517) Timeout happened during capture


In [16]:
failtrace is None

True

In [50]:
scope.trigger.window_start[0] = cycles - 1000
scope.trigger.window_end[0] = 0
goodtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [61]:
scope.trigger.window_start[0] = 0
scope.trigger.window_end[0] = cycles
goodtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12
(ChipWhisperer Scope WARNING|File __init__.py:517) Timeout happened during capture


In [60]:
scope.trigger.window_start[0] = cycles - 1000
scope.trigger.window_end[0] = cycles + 32000
goodtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))

In [40]:
scope.trigger

sequencer_enabled      = True
max_sequenced_triggers = 2
num_triggers           = 2
sad_always_active      = False
module                 = SequenceTriggerList(['basic', 'UART'])
triggers               = SequenceTriggerList(['tio4', 'tio1'])
window_start           = SequenceTriggerList([0])
window_end             = SequenceTriggerList([0])
sequence trigger #0 = 
    module   = basic
    triggers = tio4
sequence trigger #1 = 
    module       = UART
    triggers     = tio1
    window_start = 0
    window_end   = 0

In [51]:
scope.errors

sam_errors      = False
sam_led_setting = Default
XADC errors     = False
ADC errors      = ADC clipped, 
extclk error    = False
trace errors    = False

### FROM TEST_HUSKY_DEV.py:

In [4]:
scope.default_setup(verbose=False)
scope.adc.timeout = 1
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
cycles = scope.adc.trig_count
resp = target.simpleserial_last_read
reps = 1

In [15]:
scope.trigger.sequencer_enabled = True
scope.trigger.num_triggers = 2
scope.trigger.module[0] = 'basic'
scope.trigger.triggers[0] = 'tio4'
scope.trigger.module[1] = 'UART'
scope.trigger.triggers[1] = 'tio1'
scope.UARTTrigger.enabled = True
scope.UARTTrigger.baud = 38400
scope.UARTTrigger.set_pattern_match(0, resp[:4])
scope.UARTTrigger.trigger_source = 0
scope.UARTTrigger.rules_enabled = [0]

In [19]:
# first capture with no window:
scope.trigger.window_start[0] = 0
scope.trigger.window_end[0] = 0
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
assert trace is not None, 'Capture failed with no window (rep %d)' % i

In [20]:
# second capture with window:
scope.trigger.window_start[0] = cycles - 1000
scope.trigger.window_end[0] = cycles + 32000
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
assert trace is not None, 'Capture failed with window (rep %d)' % i

In [21]:
# third capture with too-restrictive window, should fail:
scope.trigger.window_start[0] = cycles//2
scope.trigger.window_end[0] = cycles
trace = cw.capture_trace(scope, target, bytearray(16), bytearray(16))
assert trace is None, 'Capture SHOULD have failed but succeeded! (rep %d)' % i

(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 13
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:732) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 12
(ChipWhisperer Scope WARNING|File __init__.py:517) Timeout happened during capture


In [ ]:
scope.trigger.sequencer_enabled = False

NameError: name 'i' is not defined

In [13]:
cycles

31864

In [14]:
scope.trigger

sequencer_enabled = False
module            = basic
triggers          = tio4